# Генеративні мережі

Рекурентні нейронні мережі (RNN) та їх варіанти з керованими осередками, такі як осередки довготривалої короткочасної пам'яті (LSTM) і керовані рекурентні блоки (GRU), забезпечили механізм для моделювання мови, тобто вони можуть навчитися впорядковувати слова та передбачати наступне слово в послідовності. Це дозволяє використовувати RNN для **генеративних завдань**, таких як звичайна генерація тексту, машинний переклад і навіть створення підписів до зображень.

У архітектурі RNN, яку ми обговорювали в попередньому розділі, кожен блок RNN генерував наступний прихований стан як вихід. Однак ми також можемо додати ще один вихід до кожного рекурентного блоку, що дозволить нам генерувати **послідовність** (яка дорівнює за довжиною оригінальній послідовності). Більше того, ми можемо використовувати блоки RNN, які не приймають вхідні дані на кожному кроці, а лише беруть початковий вектор стану і потім генерують послідовність виходів.

У цьому ноутбуці ми зосередимося на простих генеративних моделях, які допомагають нам створювати текст. Для простоти побудуємо **мережу на рівні символів**, яка генерує текст літера за літерою. Під час навчання нам потрібно взяти якийсь текстовий корпус і розбити його на послідовності літер.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Створення словника символів

Для створення генеративної мережі на рівні символів необхідно розділити текст на окремі символи, а не слова. Це можна зробити, визначивши інший токенізатор:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Давайте розглянемо приклад того, як ми можемо закодувати текст з нашого набору даних:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Навчання генеративної RNN

Метод, за яким ми будемо навчати RNN для генерації тексту, виглядає наступним чином. На кожному кроці ми беремо послідовність символів довжиною `nchars` і просимо мережу згенерувати наступний вихідний символ для кожного вхідного символу:

![Зображення, що показує приклад генерації слова 'HELLO' за допомогою RNN.](../../../../../translated_images/uk/rnn-generate.56c54afb52f9781d.webp)

Залежно від конкретного сценарію, ми також можемо захотіти включити деякі спеціальні символи, такі як *кінець послідовності* `<eos>`. У нашому випадку ми просто хочемо навчити мережу генерувати нескінченний текст, тому ми зафіксуємо розмір кожної послідовності, щоб він дорівнював `nchars` токенам. Таким чином, кожен навчальний приклад складатиметься з `nchars` входів і `nchars` виходів (які є вхідною послідовністю, зміщеною на один символ вліво). Мініпакет складатиметься з кількох таких послідовностей.

Метод генерації мініпакетів полягає в тому, щоб взяти кожен текст новини довжиною `l` і створити всі можливі комбінації вхід-вихід із нього (таких комбінацій буде `l-nchars`). Вони утворюватимуть один мініпакет, і розмір мініпакетів буде різним на кожному етапі навчання.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Тепер давайте визначимо генераторну мережу. Вона може базуватися на будь-якій рекурентній клітині, яку ми обговорювали в попередньому модулі (простій, LSTM або GRU). У нашому прикладі ми використаємо LSTM.

Оскільки мережа отримує символи як вхідні дані, а розмір словника досить малий, нам не потрібен шар вбудовування, однохот-кодований вхід може безпосередньо передаватися до клітини LSTM. Проте, оскільки ми передаємо номери символів як вхідні дані, нам потрібно закодувати їх у форматі однохот перед передачею до LSTM. Це виконується шляхом виклику функції `one_hot` під час проходу `forward`. Енкодер виходу буде лінійним шаром, який перетворює прихований стан у однохот-кодований вихід.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Під час навчання ми хочемо мати можливість вибірково генерувати текст. Для цього ми визначимо функцію `generate`, яка буде створювати вихідний рядок довжиною `size`, починаючи з початкового рядка `start`.

Ось як це працює. Спочатку ми передаємо весь початковий рядок через мережу, отримуємо вихідний стан `s` і наступний передбачений символ `out`. Оскільки `out` закодований у форматі one-hot, ми використовуємо `argmax`, щоб отримати індекс символу `nc` у словнику, і за допомогою `itos` визначаємо фактичний символ, який додаємо до списку результативних символів `chars`. Цей процес генерації одного символу повторюється `size` разів, щоб створити необхідну кількість символів.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Тепер давайте перейдемо до тренування! Цикл тренування майже такий самий, як у всіх наших попередніх прикладах, але замість точності ми виводимо згенерований текст кожні 1000 епох.

Особливу увагу потрібно приділити способу обчислення втрат. Нам потрібно обчислити втрати, враховуючи одновимірний закодований вихід `out` і очікуваний текст `text_out`, який є списком індексів символів. На щастя, функція `cross_entropy` очікує на вхід ненормалізований вихід мережі як перший аргумент і номер класу як другий, що саме відповідає нашим даним. Вона також автоматично виконує усереднення за розміром мініпакету.

Ми також обмежуємо тренування кількістю зразків `samples_to_train`, щоб не чекати надто довго. Ми заохочуємо вас експериментувати та спробувати довше тренування, можливо, протягом кількох епох (у такому випадку вам потрібно буде створити ще один цикл навколо цього коду).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Цей приклад вже генерує досить якісний текст, але його можна покращити кількома способами:

* **Краща генерація мініпакетів**. Спосіб, яким ми підготували дані для навчання, полягав у створенні одного мініпакета з одного зразка. Це не ідеально, оскільки мініпакети мають різний розмір, а деякі з них взагалі неможливо згенерувати, якщо текст менший за `nchars`. Крім того, маленькі мініпакети недостатньо завантажують GPU. Було б розумніше взяти великий фрагмент тексту з усіх зразків, потім згенерувати всі пари вхід-вихід, перемішати їх і створити мініпакети однакового розміру.

* **Багатошаровий LSTM**. Варто спробувати 2 або 3 шари LSTM-комірок. Як ми згадували в попередньому розділі, кожен шар LSTM витягує певні шаблони з тексту, і у випадку генератора на рівні символів можна очікувати, що нижчий рівень LSTM відповідатиме за витягування складів, а вищі рівні — за слова та їх комбінації. Це можна легко реалізувати, передавши параметр кількості шарів у конструктор LSTM.

* Ви також можете поекспериментувати з **GRU-комірками** і подивитися, які з них працюють краще, а також з **різними розмірами прихованих шарів**. Занадто великий прихований шар може призвести до перенавчання (наприклад, мережа запам'ятає точний текст), а менший розмір може не дати хорошого результату.


## Генерація м'якого тексту та температура

У попередньому визначенні `generate` ми завжди обирали символ із найвищою ймовірністю як наступний символ у згенерованому тексті. Це призводило до того, що текст часто "зациклювався" на одних і тих самих послідовностях символів знову і знову, як у цьому прикладі:
```
today of the second the company and a second the company ...
```

Однак, якщо подивитися на розподіл ймовірностей для наступного символу, може виявитися, що різниця між кількома найвищими ймовірностями не є значною, наприклад, один символ може мати ймовірність 0.2, а інший — 0.19 тощо. Наприклад, коли шукаємо наступний символ у послідовності '*play*', наступним символом може однаково бути пробіл або **e** (як у слові *player*).

Це приводить нас до висновку, що не завжди "справедливо" обирати символ із вищою ймовірністю, оскільки вибір другого за величиною ймовірності також може привести до осмисленого тексту. Більш розумно **вибирати** символи з розподілу ймовірностей, який надає вихід мережі.

Це вибіркове обрання можна здійснити за допомогою функції `multinomial`, яка реалізує так званий **мультиноміальний розподіл**. Функція, яка реалізує цю **м'яку** генерацію тексту, визначена нижче:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Ми ввели ще один параметр, який називається **temperature** (температура), і він використовується для визначення того, наскільки сильно ми повинні дотримуватися найвищої ймовірності. Якщо температура дорівнює 1.0, ми виконуємо справедливе мультиноміальне вибіркове моделювання, а коли температура наближається до нескінченності - всі ймовірності стають рівними, і ми випадково вибираємо наступний символ. У наведеному нижче прикладі ми можемо спостерігати, що текст стає безглуздим, коли ми занадто сильно збільшуємо температуру, і він нагадує "циклічний" текст, створений з жорсткими обмеженнями, коли температура наближається до 0.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
